In [1]:
import cv2
import mediapipe as mp
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [2]:
# Initialize MediaPipe Pose and Drawing utilities
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

In [3]:
# Action List
CLASSES_LIST = ['Jump',
 'Left High Kick',
 #'Idle',
 'Punch Left',
 #'Uppercut',
 'Block',
 'Duck',
 #'Stance',
 'Punch Right',
 'Right Low Kick',
 'Right High Kick',
 'Left Low Kick']

In [4]:
max_num_frames = 15  # maximum number of frames per action sequence
num_landmarks = 23    # number of landmarks per frame
num_features = 4      # number of features per landmark (x, y, z, visibility)
num_classes = len(CLASSES_LIST)       # number of action classes

In [5]:
model = tf.keras.models.load_model('models/model2v2.h5')

In [6]:
def normalize_z_values_for_realtime(landmarks):
    """
    Normalize the z-values of all pose landmarks(0-33) to a range of [0, 1].

    Parameters:
    landmarks (list): List of pose landmarks, each represented as [x, y, z, visibility].

    Returns:
    list: Normalized pose landmarks with z-values scaled to [0, 1].
    """
    if not landmarks: return landmarks

    min_z = -2
    max_z = 2

    
    # Normalize z-values to range [0, 1]
    normalized_landmarks = []
    for lm in landmarks:
        if max_z - min_z != 0:
            normalized_z = (lm[2] - min_z) / (max_z - min_z)
        else:
            normalized_z = lm[2]  # Handle edge case
        # Replace with the desired visibility value
        
        normalized_landmarks.append([lm[0], lm[1], normalized_z, lm[3]])  # Keep x, y, visibility unchanged
        # else:
        #     normalized_landmarks.append([-1,-1,-1,-1])

    return normalized_landmarks

In [7]:
def predict_action(sequence):
    # Pad the sequence
    padded_sequence = pad_sequences([sequence], maxlen=max_num_frames, dtype='float32', padding='post', truncating='post', value=-1)
    padded_sequence = padded_sequence.reshape((padded_sequence.shape[0], padded_sequence.shape[1], num_landmarks * num_features))
    
    # Predict the action
    predictions = model.predict(padded_sequence)
    return np.argmax(predictions), np.max(predictions)

In [11]:
#cap = cv2.VideoCapture('D:\Programming\AI\Open CV\Mediapipe\Final_Dataset_trim\Kicking\kick_right10 - Trim.mp4')
# Start capturing video from webcam
cap = cv2.VideoCapture(0)
# Initialize a sequence to store landmarks
sequence = []
exclude_points = [1,2,3,4,5,6,7,8,9,10]
action_label = ''
confidence =0
visibility_threshold = 0.8
with mp_pose.Pose(model_complexity=0,min_detection_confidence=0.9, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Convert the image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Perform pose detection
        results = pose.process(image)

        # Convert the image back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        
        # Draw the pose annotation on the image
        if results.pose_landmarks:

            # visible_landmarks_count = 0
            # # Calculate the number of visible landmarks. We skip 0 to 10 because they are facial points and are not required
            # for idx,l in enumerate(results.pose_landmarks.landmark):
            #     if idx>10 and l.visibility > visibility_threshold:
            #         visible_landmarks_count += 1
       
            # # Only draw the landmarks if more that 20 landmarks are detected
            # if visible_landmarks_count < 20: 
            #     cv2.putText(image, f'Only {visible_landmarks_count} points visible', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)        
            #     continue                                       
            mp_drawing.draw_landmarks(
                image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            temp_csv = []       
            for idx,landmark in enumerate(results.pose_landmarks.landmark):                
                if idx not in exclude_points:                            
                    #print(f"{mp_pose.PoseLandmark(idx).name}: (x: {landmark.x}, y: {landmark.y}, z: {landmark.z})")
                    temp_csv.append([landmark.x, landmark.y, landmark.z,landmark.visibility]) 
            normalized_landmarks = normalize_z_values_for_realtime(temp_csv)
            
            # write_landmarks_to_csv_temp(normalized_landmarks, framess, csv_data2)
            if len(normalized_landmarks) > 0:
                sequence.append(normalized_landmarks)
            
            #If sequence is too long, remove the oldest frame
            if len(sequence) > max_num_frames:
                sequence.pop(0)
            #cv2.putText(image, f'{len(sequence)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            #Make a prediction if sequence has enough frames
            if len(sequence) == max_num_frames:
                action_idx, confidence = predict_action(sequence)
                action_label = CLASSES_LIST[action_idx]
                if confidence>0.8:
                    sequence = []                                
            
            if confidence> 0.85:            
                cv2.putText(image, f'Action: {action_label} ({confidence:.2f})', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)        
        
        # Display the resulting frame
        cv2.imshow('Webcam Action Recognition', image)
        
        # Break the loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # with open(f'outfile', mode='w', newline='') as file:
    #     writer = csv.writer(file)
    #     # Write each row of the 2D array
    #     for row in csv_data2:
    #         writer.writerow(row)
    # Release the capture
    cap.release()
    cv2.destroyAllWindows()

c:\Users\Ankit\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 [==============================] - 0s 24ms/step
